<a href="https://colab.research.google.com/github/saktheeswaranswan/random-math-addition-data-for-rnn-generator/blob/main/shutup_pannuga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import csv
import librosa

# Load audio file
audio_file = '/content/hornc.wav'
audio_data, sampling_rate = librosa.load(audio_file, sr=None)

# Get time vector for audio
duration = librosa.get_duration(audio_data, sr=sampling_rate)
time = np.linspace(0, duration, len(audio_data))

# Save waveform to CSV file
with open('audio_waveform.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Time', 'Amplitude'])
    for i in range(len(time)):
        writer.writerow([time[i], audio_data[i]])


In [4]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import csv
import numpy as np
from scipy.io import wavfile
from pydub import AudioSegment

# Convert MP3 file to WAV
AudioSegment.from_mp3("/content/sample_data/dfhdfgh.mp3").export("audio.wav", format="wav")

# Read CSV file
with open('/content/audio_waveform.csv', 'r') as file:
    reader = csv.reader(file)
    # Skip header row if present
    next(reader, None)
    # Extract time and amplitude values from each row
    time = []
    amplitude = []
    for row in reader:
        time.append(float(row[0]))
        amplitude.append(float(row[1]))

# Calculate sampling rate and duration of audio
sampling_rate, audio_data = wavfile.read('audio.wav')
duration = len(audio_data) / sampling_rate

# Create time vector for audio
audio_time = np.linspace(0, duration, len(audio_data))

# Interpolate waveform data to match audio sampling rate
interpolated_amplitude = np.interp(audio_time, time, amplitude)

# Invert waveform to create an "anti-signal"
anti_signal = -1 * interpolated_amplitude

# Shift anti-signal to align with audio
delay_in_seconds = 1.0  # adjust as needed
shifted_anti_signal = np.roll(anti_signal, int(sampling_rate * delay_in_seconds))

# Silence audio by multiplying it with the shifted anti-signal
silenced_audio = audio_data * shifted_anti_signal

# Write silenced audio to new WAV file
wavfile.write('silenced_audio.wav', sampling_rate, silenced_audio)


ValueError: ignored

In [8]:
import csv
import numpy as np
from scipy.io import wavfile
from pydub import AudioSegment

# Convert MP3 file to WAV
AudioSegment.from_mp3("/content/horn.wav").export("audio.wav", format="wav")

# Read CSV file
with open('audio_waveform.csv', 'r') as file:
    reader = csv.reader(file)
    # Skip header row if present
    next(reader, None)
    # Extract amplitude values from second column of each row
    amplitude = []
    for row in reader:
        amplitude.append(float(row[1]))

# Calculate sampling rate and duration of audio
sampling_rate, audio_data = wavfile.read('audio.wav')
duration = len(audio_data) / sampling_rate

# Create time vector for audio
audio_time = np.linspace(0, duration, len(audio_data))

# Interpolate waveform data to match audio sampling rate
interpolated_amplitude = np.interp(audio_time, audio_time, amplitude)

# Invert waveform to create an "anti-signal"
anti_signal = -1 * interpolated_amplitude

# Shift anti-signal to align with audio
delay_in_seconds = 1.0  # adjust as needed
shifted_anti_signal = np.roll(anti_signal, int(sampling_rate * delay_in_seconds))

# Silence audio by multiplying it with the shifted anti-signal
silenced_audio = audio_data * shifted_anti_signal[:, np.newaxis]

# Write silenced audio to new WAV file
wavfile.write('silenced_audio.wav', sampling_rate, silenced_audio)


In [11]:
import csv
import numpy as np
from scipy.io import wavfile
from pydub import AudioSegment

# Convert MP3 file to WAV
AudioSegment.from_mp3("/content/hornc.wav").export("audio.wav", format="wav")

# Read CSV file
with open('audio_waveform.csv', 'r') as file:
    reader = csv.reader(file)
    # Skip header row if present
    next(reader, None)
    # Extract time and amplitude values from each row
    time = []
    amplitude = []
    for row in reader:
        time.append(float(row[0]))
        amplitude.append(float(row[1]))

# Calculate sampling rate and duration of audio
sampling_rate, audio_data = wavfile.read('audio.wav')
duration = len(audio_data) / sampling_rate

# Create time vector for audio
audio_time = np.linspace(0, duration, len(audio_data))

# Interpolate waveform data to match audio sampling rate
interpolated_amplitude = np.interp(audio_time, time, amplitude)

# Create a zero amplitude waveform
silence = np.zeros_like(interpolated_amplitude)

# Shift silence waveform to align with audio
delay_in_seconds = 1.0  # adjust as needed
shifted_silence = np.roll(silence, int(sampling_rate * delay_in_seconds))

# Silence audio by multiplying it with the shifted silence waveform
silenced_audio = audio_data * shifted_silence[:, np.newaxis]

# Write silenced audio to new WAV file
wavfile.write('silenced_audio.wav', sampling_rate, silenced_audio)


In [ ]:
import numpy as np
import pyaudio
import queue

# Define constants
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
DELAY = 1.0

# Initialize PyAudio
p = pyaudio.PyAudio()

# Open microphone stream
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK_SIZE)

# Define a queue to store audio chunks
queue = queue.Queue()

# Start streaming audio
def callback(in_data, frame_count, time_info, status):
    queue.put(in_data)
    return (None, pyaudio.paContinue)

stream.start_stream()
stream.set_callback(callback)

# Read waveform data from file
waveform_data = np.genfromtxt('audio_waveform.csv', delimiter=',', skip_header=1)

# Interpolate waveform data to match audio sampling rate
audio_time = np.linspace(0, CHUNK_SIZE/RATE, CHUNK_SIZE)
interpolated_waveform = np.interp(audio_time, waveform_data[:,0], waveform_data[:,1])

# Invert waveform to create an "anti-signal"
anti_signal = -1 * interpolated_waveform

# Shift anti-signal to align with audio
delay_in_samples = int(RATE * DELAY)
shifted_anti_signal = np.roll(anti_signal, delay_in_samples)

# Apply silencing in real time
while True:
    # Get audio chunk from queue
    audio_chunk = queue.get()
    audio_data = np.frombuffer(audio_chunk, dtype=np.int16)

    # Apply silence to audio
    silenced_audio = audio_data * shifted_anti_signal[:len(audio_data)]

    # Convert silenced audio back to bytes and write to output stream
    stream.write(silenced_audio.tobytes())
